# Imports

In [ ]:
# !CFLAGS="-I/usr/include" LDFLAGS="-L/usr/lib -lenet" pip install pyenet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.4/498.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.7 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.12
    Uninstalling Cython-3.0.12:
      Successfully uninstalled Cython-3.0.12


In [ ]:
# !pip install git+https://github.com/vladfi1/libmelee.git

  Cloning https://github.com/vladfi1/libmelee.git to /tmp/pip-req-build-vp6zzys8
  Running command git clone --filter=blob:none --quiet https://github.com/vladfi1/libmelee.git /tmp/pip-req-build-vp6zzys8
  Resolved https://github.com/vladfi1/libmelee.git to commit 60981aca415c27d284ef51cc3e3c22b68e6ed8cb
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/piqueserver/pyenet to /tmp/pip-install-0zki0pk3/pyenet_eb8261dd69b541368bffc6563270b287
  Running command git clone --filter=blob:none --quiet https://github.com/piqueserver/pyenet /tmp/pip-install-0zki0pk3/pyenet_eb8261dd69b541368bffc6563270b287
  Resolved https://github.com/piqueserver/pyenet to commit 1bd4e84b4d6bcfb171c11572a1b7b770123e3771
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for melee: filename=melee-0.38.0-py3-n

In [ ]:
# !apt-get update
# !apt-get install -y cmake libenet-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [75.2 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,604 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/mai

In [4]:
# !pip install peppi_py

In [5]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [105]:
import os
import melee
import numpy as np
import glob
import torch
import torch.nn as nn
from torch.utils.data import Dataset, ConcatDataset

# Data Processing (Deprecated)

In [8]:
os.getcwd()

'/content'

In [9]:
# console = melee.Console(is_dolphin=False,
#                         allow_old_version=False,
#                         path="./data/BTSSmash-Game_20200126T202356.slp"
#                         )
# console.connect()

# while True:
#     gamestate = console.step()
#     # step() returns None when the file ends
#     if gamestate is None:
#         break
#     print("Frame " + str(gamestate.frame))
#     for _, player in gamestate.players.items():
#         print("\t", player.stock, player.percent)

In [10]:
# import peppi_py as peppi
# import numpy as np

# def get_numpy_row(pre, post):
#     def safe_extract(val):
#         return val.to_numpy() # if val is not None else np.nan

#     features = []

#     # --- Pre ---
#     features += [
#         safe_extract(pre.position.x),
#         safe_extract(pre.position.y),
#         safe_extract(pre.joystick.x),
#         safe_extract(pre.joystick.y),
#         safe_extract(pre.triggers),
#         safe_extract(pre.buttons),
#         safe_extract(pre.buttons_physical),
#         safe_extract(pre.direction),
#         safe_extract(pre.percent) if pre.percent else np.nan
#     ]

#     # --- Post ---
#     features += [
#         safe_extract(post.position.x),
#         safe_extract(post.position.y),
#         safe_extract(post.percent),
#         safe_extract(post.shield),
#         safe_extract(post.combo_count),
#         safe_extract(post.state),
#         safe_extract(post.airborne) if post.airborne else np.nan,
#         safe_extract(post.stocks),
#         safe_extract(post.last_hit_by),
#         safe_extract(post.character),
#         safe_extract(post.direction),
#     ]

#     return features

# def slp_to_numpy(path, num_ports=2):
#     game = peppi.read_slippi(path)
#     rows = []

#     for port in range(num_ports):
#         frame = game.frames
#         try:
#             pre = frame.ports[port].leader.pre
#             post = frame.ports[port].leader.post
#             row = get_numpy_row(pre, post)
#             rows.append(row)
#         except Exception:
#             # If frame/port is missing or broken, skip or pad
#             rows.append([np.nan] * 20)

#     arr = np.array(rows)
#     # transpose to (12158, 2, 20)
#     arr = arr.transpose(2, 0, 1)
#     return arr.reshape(arr.shape[0], -1)

# # Example usage
# np_data = slp_to_numpy("./drive/MyDrive/icl_smash-main/data/Stream-Game_20220828T225335.slp")
# print("Shape:", np_data.shape)
# print(np_data[:5])


In [11]:
np.save("training_data.npy", np_data)

NameError: name 'np_data' is not defined

In [ ]:
# import numpy as np
# from collections import defaultdict
# import json

# ### Step 0: Load Raw Data ###
# raw_data = np.load('./drive/MyDrive/icl_smash-main/training_data/[libmelee]example1_sampled.npy', allow_pickle=True)

# ### Step 1: Flatten a Single Frame ###
# def flatten_frame(frame):
#     flat = {}
#     for i, item in enumerate(frame):
#         key = f"col_{i}"
#         if isinstance(item, dict):  # like <Button.BUTTON_A: 'A'>: True
#             for k, v in item.items():
#                 new_key = f"{key}_{str(k)}"
#                 flat[new_key] = float(v) if isinstance(v, bool) else v
#         else:
#             flat[key] = item
#     return flat

# ### Step 2: Flatten Entire Dataset ###
# flat_data = [flatten_frame(frame) for frame in raw_data]

# ### Step 3: Build Vocab for Categorical/String Fields ###
# def build_vocab(flat_data):
#     vocab = defaultdict(set)
#     for row in flat_data:
#         for k, v in row.items():
#             if isinstance(v, str) or str(v) == 'nan':
#                 vocab[k].add(v)
#     return {k: {val: i for i, val in enumerate(sorted(list(vals)))} for k, vals in vocab.items()}

# vocab = build_vocab(flat_data)

# ### Step 4: Encode Each Frame to Numeric ###
# def encode_flat_row(row, vocab):
#     encoded = []
#     for k, v in row.items():
#         if k in vocab:
#             encoded.append(vocab[k].get(v, 0))
#         else:
#             try:
#                 encoded.append(float(v))
#             except:
#                 encoded.append(0.0)
#     return encoded

# encoded_data = np.array([encode_flat_row(row, vocab) for row in flat_data], dtype=np.float32)
# print(f"Encoded data shape: {encoded_data.shape}")  # Should be (842, <num_features>)

# ### Step 5: Sliding Window to Create Sequences ###
# def create_sequences(data, input_len=10, target_len=5):
#     input_seqs = []
#     target_seqs = []
#     for t in range(len(data) - input_len - target_len): # TODO: Currently slides 1 timestep at a time, but if we are going to infer M of them, can have the inferred M be the last M frames in the next window input
#         input_seqs.append(data[t : t + input_len]) # say, frames 10-19, t=10
#         target_seqs.append(data[t + input_len : t + input_len + target_len]) # say, frames 20-24, target_len = 5
#     return np.array(input_seqs), np.array(target_seqs)

# input_len = 10
# target_len = 5
# input_seqs, target_seqs = create_sequences(encoded_data, input_len, target_len)

# print("Input shape:", input_seqs.shape)     # (num_samples, 10, num_features)
# print("Target shape:", target_seqs.shape)   # (num_samples, 5, num_features)

# ### Step 6 (Optional): Save Output ###
# np.save("melee_input_seqs.npy", input_seqs)
# np.save("melee_target_seqs.npy", target_seqs)

# # Save vocab so you can decode predictions later
# with open("melee_vocab.json", "w") as f:
#     json.dump({k: dict(v) for k, v in vocab.items()}, f)

# print("✅ Preprocessing complete.")


# EDA

In [157]:
import importlib
import data_processing
importlib.reload(data_processing)
from data_processing import main
main("./data/(...) Falco vs Falco (BABE) [BF] Game_20200122T213211.slp", "./old_data", "viewable")

(637, 59)

In [163]:
# Load viewable data and columns
viewable_data = np.load("./old_data/viewable_data.npy", allow_pickle=True)
columns = np.load("./old_data/viewable_columns.npy", allow_pickle=True)

# Create DataFrame
df = pd.DataFrame(viewable_data.astype('float32'), columns=columns)

# Display basic information
print("DataFrame shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

DataFrame shape: (637, 59)

Columns: ['frame', 'distance', 'p1_action_frame', 'p1_controller_c_stick_x', 'p1_controller_c_stick_y', 'p1_controller_l_shoulder', 'p1_controller_r_shoulder', 'p1_controller_main_stick_x', 'p1_controller_main_stick_y', 'p1_button_button_a', 'p1_button_button_b', 'p1_button_button_x', 'p1_button_button_y', 'p1_button_button_z', 'p1_button_button_l', 'p1_button_button_r', 'p1_button_button_start', 'p1_facing', 'p1_hitlag_left', 'p1_hitstun_frames_left', 'p1_jumps_left', 'p1_off_stage', 'p1_on_ground', 'p1_percent', 'p1_position_x', 'p1_position_y', 'p1_shield_strength', 'p1_stock', 'p3_action_frame', 'p3_controller_c_stick_x', 'p3_controller_c_stick_y', 'p3_controller_l_shoulder', 'p3_controller_r_shoulder', 'p3_controller_main_stick_x', 'p3_controller_main_stick_y', 'p3_button_button_a', 'p3_button_button_b', 'p3_button_button_x', 'p3_button_button_y', 'p3_button_button_z', 'p3_button_button_l', 'p3_button_button_r', 'p3_button_button_start', 'p3_facing', 'p

In [164]:
for column in df.columns:
    print(df[column].value_counts(), "\n")

frame
-123.0     1
 6297.0    1
 6192.0    1
 6207.0    1
 6222.0    1
          ..
 3072.0    1
 3087.0    1
 3102.0    1
 3117.0    1
 9417.0    1
Name: count, Length: 637, dtype: int64 

distance
0.0    637
Name: count, dtype: int64 

p1_action_frame
 1.0     61
 4.0     48
 2.0     47
 3.0     43
 5.0     37
-1.0     35
 13.0    32
 9.0     27
 6.0     23
 11.0    20
 7.0     20
 8.0     17
 10.0    16
 16.0    15
 20.0    12
 19.0    12
 29.0    11
 12.0    11
 15.0    11
 17.0    11
 0.0     10
 14.0    10
 28.0     9
 25.0     8
 27.0     8
 26.0     8
 35.0     6
 22.0     6
 23.0     6
 18.0     5
 24.0     4
 31.0     4
-2.0      4
 41.0     4
 40.0     4
 32.0     4
 30.0     3
 38.0     3
 21.0     3
 33.0     2
 34.0     2
 37.0     2
 36.0     2
 47.0     1
 48.0     1
 58.0     1
 43.0     1
 55.0     1
 42.0     1
 53.0     1
 49.0     1
 57.0     1
 52.0     1
 50.0     1
Name: count, dtype: int64 

p1_controller_c_stick_x
0.50000    613
0.00625      3
0.26875      3
0

# Define Constants and Parameters

In [160]:
DATA_FOLDER_PATH = "./training_data"
NUM_ENUM_COLUMNS = 5  # Last 5 columns of the numpy arrays

# Data Loading and Model Training

In [165]:
import importlib
import data_loading
importlib.reload(data_loading) 
from data_loading import BatchedFilesDataset, MeleeDataset
import pandas as pd

In [166]:
# Load the data
data = np.load("./training_data/(...) Falco vs Falco (BABE) [BF] Game_20200122T213211_sequences.npz", allow_pickle=True)
inputs = data['inputs']

# Find columns with non-numeric data
non_numeric_cols = []
for col in columns:
    if not np.issubdtype(df[col].dtype, np.number):
        print(f"Column '{col}' has non-numeric type: {df[col].dtype}")
        print(f"Sample values: {df[col].head()}\n")
        non_numeric_cols.append(col)

print(f"\nTotal non-numeric columns found: {len(non_numeric_cols)}")
print(f"Non-numeric column names: {non_numeric_cols}")


Total non-numeric columns found: 0
Non-numeric column names: []


In [167]:
dataset = MeleeDataset(data_path="./training_data/(...) Falco vs Falco (BABE) [BF] Game_20200122T213211_sequences.npz", match_id=0, num_enums=5)

Original input shape: (623, 59, 10)
Transposed input shape: (623, 10, 59)


In [156]:
batched_files = BatchedFilesDataset(DATA_FOLDER_PATH, files_per_batch=2)

batched_files.load_next_batch()
dataloader = DataLoader(batched_files.load_next_batch(), batch_size=64, shuffle=True)

Original input shape: (690, 59, 10)
Transposed input shape: (690, 10, 59)
Loaded ./training_data/(...) Falco vs Falco (BABE) [DL] Game_20200122T213501_sequences.npz
Original input shape: (623, 59, 10)
Transposed input shape: (623, 10, 59)
Loaded ./training_data/(...) Falco vs Falco (BABE) [BF] Game_20200122T213211_sequences.npz
[<data_loading.MeleeDataset object at 0x146230bb0>, <data_loading.MeleeDataset object at 0x1598c1360>]
Original input shape: (460, 59, 10)
Transposed input shape: (460, 10, 59)
Loaded ./training_data/(...) Falco vs Falcon (L$) [YS] Game_20200226T210843_sequences.npz


In [ ]:
class MeleeEncoderDecoder(nn.Module):
    def __init__(self, continuous_dim, enum_embed_dim, d_model=128, nhead=4, num_layers=3):
        super().__init__()
        self.enum_embedder = EnumEmbeddingModule()
        self.encoder_proj = nn.Linear(continuous_dim + enum_embed_dim, d_model)
        self.decoder_proj = nn.Linear(continuous_dim + enum_embed_dim, d_model)

        self.pos_enc = nn.Parameter(torch.randn(1, 100, d_model))  # fixed max seq_len

        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead)
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead)

        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers)

        self.output_proj = nn.Linear(d_model, continuous_dim)  # only predicting continuous features
        self.enum_embed_dim = enum_embed_dim

    def forward(self, src_cont, src_enum, tgt_cont, tgt_enum):
        # src_cont, tgt_cont: (batch, seq_len, continuous_dim)
        # src_enum, tgt_enum: tuple of (character_ids, stage_ids, etc.)

        # Step 1: Embed enums
        src_enum_embed = self.enum_embedder(*src_enum)  # (batch, seq_len, enum_embed_dim)
        tgt_enum_embed = self.enum_embedder(*tgt_enum)

        # Step 2: Concatenate continuous + enum embeddings
        src = torch.cat([src_cont, src_enum_embed], dim=-1)  # (batch, seq_len, continuous + enum dim)
        tgt = torch.cat([tgt_cont, tgt_enum_embed], dim=-1)

        # Step 3: Positional encoding + projection
        src = self.encoder_proj(src) + self.pos_enc[:, :src.size(1), :]
        tgt = self.decoder_proj(tgt) + self.pos_enc[:, :tgt.size(1), :]

        # Step 4: Transformer (permute to seq_len, batch, d_model)
        src = src.permute(1, 0, 2)
        tgt = tgt.permute(1, 0, 2)

        memory = self.encoder(src)
        output = self.decoder(tgt, memory)

        return self.output_proj(output.permute(1, 0, 2))  # (batch, tgt_len, continuous_dim)

In [ ]:
# model = MeleeEncoderDecoder(feature_dim=dataset.inputs.shape[-1])
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# loss_fn = nn.MSELoss()

# for epoch in range(10):
#     model.train()
#     total_loss = 0

#     for x_batch, y_batch in dataloader: # frames 10-19, and 20-24
#         decoder_input = y_batch[:, :-1, :]  # teacher forcing; grabs everything up until the last frame of the sequence, 20-23
#         decoder_target = y_batch[:, 1:, :]  # grabs everything after the 1st frame, 21-24

#         preds = model(x_batch, decoder_input) # given frames 10-19 and the decoder input of 20-23, output is which frames?

#         loss = loss_fn(preds, decoder_target)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.4f}")


In [ ]:
import torch
import torch.nn as nn
import wandb

# Initialize a W&B run
wandb.init(
    project="slp_test",    # ⬅️ replace with your project
    name="melee-encoder-decoder",   # ⬅️ optional run name
    config={                        # ⬅️ log your hyperparameters
        "lr": 1e-4,
        "epochs": 10,
        "batch_size": dataloader.batch_size if hasattr(dataloader, "batch_size") else None,
        "feature_dim": dataset.inputs.shape[-1],
    }
)
config = wandb.config

# Instantiate your model, optimizer, loss
# model = MeleeEncoderDecoder(feature_dim=config.feature_dim)
# optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
loss_fn = nn.MSELoss()

# Suppose dataset.inputs is shape [N, seq_len, feat_dim]
all_x = dataset.inputs.view(-1, dataset.inputs.size(-1))   # [N*seq_len, feat_dim]
all_y = dataset.targets.view(-1, dataset.targets.size(-1))

x_mean, x_std = all_x.mean(0), all_x.std(0)
y_mean, y_std = all_y.mean(0), all_y.std(0)

# Normalize
def normalize(x, mean, std):
    return (x - mean) / (std + 1e-6)

# Compute normalization stats once:
all_x = dataset.inputs.view(-1, dataset.inputs.size(-1))
all_y = dataset.targets.view(-1, dataset.targets.size(-1))
x_mean, x_std = all_x.mean(0), all_x.std(0)
y_mean, y_std = all_y.mean(0), all_y.std(0)

# Build model & optimizer with larger lr:
model = MeleeEncoderDecoder(feature_dim=dataset.inputs.shape[-1])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       mode="min",
                                                       factor=0.5,
                                                       patience=3)

# Tell W&B to watch the model (logs gradients & parameter histograms)
wandb.watch(model, log="all", log_freq=100)


for epoch in range(20):
    model.train()
    total_loss = 0

    for x_batch, y_batch in dataloader:
        # normalize
        x_batch = (x_batch - x_mean) / (x_std + 1e-6)
        y_batch = (y_batch - y_mean) / (y_std + 1e-6)

        decoder_input  = y_batch[:, :-1, :]
        decoder_target = y_batch[:,  1:, :]

        preds_norm = model(x_batch, decoder_input)
        loss = loss_fn(preds_norm, decoder_target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    # step scheduler
    scheduler.step(avg_loss)

# # 6. Optionally, save your final model as an artifact
# artifact = wandb.Artifact("melee-encoder-decoder-model", type="model")
# artifact.add_file("path/to/your/saved_model.pt")
# wandb.log_artifact(artifact)

# 7. Finish the run
wandb.finish()


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1, Loss: 0.7485
Epoch 2, Loss: 0.6757
Epoch 3, Loss: 0.6292
Epoch 4, Loss: 0.5674
Epoch 5, Loss: 0.5100
Epoch 6, Loss: 0.4503
Epoch 7, Loss: 0.3926
Epoch 8, Loss: 0.3438
Epoch 9, Loss: 0.3044
Epoch 10, Loss: 0.2712
Epoch 11, Loss: 0.2453
Epoch 12, Loss: 0.2231
Epoch 13, Loss: 0.2039
Epoch 14, Loss: 0.1884
Epoch 15, Loss: 0.1745
Epoch 16, Loss: 0.1627
Epoch 17, Loss: 0.1518
Epoch 18, Loss: 0.1436
Epoch 19, Loss: 0.1333
Epoch 20, Loss: 0.1255


In [ ]:
import torch

# suppose you still have y_mean, y_std from training,
# both of shape (feature_dim,)
# and you moved them to the same device as your model:

device    = next(model.parameters()).device
y_mean    = y_mean.to(device)    # shape (feature_dim,)
y_std     = y_std.to(device)     # shape (feature_dim,)

model.eval()
with torch.no_grad():
    # grab one test input
    input_seq = dataset.inputs[-1].unsqueeze(0).to(device)  # (1, in_len, feat_dim)
    # start decoder with one “zero” frame
    decoder_input = torch.zeros((1, 1, dataset.inputs.shape[-1]),
                                device=device)

    preds_norm = []
    for _ in range(5):
        out = model(input_seq, decoder_input)     # (1, cur_tgt_len, feat_dim)
        next_step = out[:, -1:, :]                # (1, 1, feat_dim)
        preds_norm.append(next_step)
        decoder_input = torch.cat([decoder_input, next_step], dim=1)

    preds_norm = torch.cat(preds_norm, dim=1)     # (1, 5, feat_dim)

    # --- Undo the scaling ---
    # reshape mean/std to broadcast over (batch, time, feature)
    mu = y_mean.view(1, 1, -1)    # (1,1,feat_dim)
    sigma = y_std.view(1, 1, -1)  # (1,1,feat_dim)

    preds_denorm = preds_norm * sigma + mu      # (1, 5, feat_dim)

    # now preds_denorm is in the original units
    # you can convert to NumPy if you like:
    preds_denorm = preds_denorm.squeeze(0).cpu().numpy()  # (5, feat_dim)

print(preds_denorm.shape)  # e.g. (5, feat_dim)


(5, 119)
